In [1]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
len(rc)

17

In [2]:
rc.purge_everything() #Limpiamos la sesion

In [3]:
dview = rc[:]
bview = rc.load_balanced_view()# targets=list(range(40))
dview

<DirectView [0, 1, 2, 3,...]>

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv("Data/kc_house_data.csv")
#revisando las columnas del dataset
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [6]:
columns = list(data.columns)
columns.remove('id')
columns.remove('date')
columns.remove('floors')
columns.remove('waterfront')
columns.remove('view')
columns.remove('condition')
#columns.remove('grade')
columns.remove('zipcode')

floors = pd.get_dummies(data.floors,prefix="floors")
waterfront = pd.get_dummies(data.waterfront,prefix="waterfront")
view = pd.get_dummies(data.view,prefix="view")
condition = pd.get_dummies(data.condition,prefix="condition")
grade = pd.get_dummies(data.grade,prefix="grade")
zipcode = pd.get_dummies(data.zipcode,prefix="zipcode")

data_pre = data[columns]
data_pre = data_pre.join(floors)
data_pre = data_pre.join(waterfront)
data_pre = data_pre.join(view)
data_pre = data_pre.join(condition)
#data_pre = data_pre.join(grade)
data_pre = data_pre.join(zipcode)
data_pre["price"] = data["price"]
data_pre.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,yr_built,yr_renovated,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,221900.0,3,1.00,1180,5650,7,1180,0,1955,0,...,0,0,0,0,0,0,1,0,0,0
1,538000.0,3,2.25,2570,7242,7,2170,400,1951,1991,...,0,0,0,0,0,0,0,0,0,0
2,180000.0,2,1.00,770,10000,6,770,0,1933,0,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,5000,7,1050,910,1965,0,...,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,1680,8080,8,1680,0,1987,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
tr_df, temp_df = train_test_split(data_pre, test_size=0.5,random_state=7)
val_df, test_df = train_test_split(temp_df, test_size=0.4,random_state=7)
print("Train(%): ", len(tr_df)*100/len(data_pre))
print("Val(%):   ", len(val_df)*100/len(data_pre))
print("Test(%):  ", len(test_df)*100/len(data_pre))

Train(%):  49.997686577522785
Val(%):    30.000462684495442
Test(%):   20.00185073798177


In [9]:
tr_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,yr_built,yr_renovated,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
17649,920000.0,4,3.25,2420,4000,9,1870,550,1911,0,...,0,0,0,0,0,0,0,0,0,0
9742,999950.0,3,2.25,3740,22464,8,2330,1410,1966,0,...,0,0,0,0,0,0,0,0,0,0
4613,880000.0,4,2.75,2560,7961,8,1450,1110,1975,0,...,0,0,0,0,0,0,0,0,0,0
19347,535000.0,4,1.75,1420,5000,7,1420,0,1945,0,...,0,0,0,0,0,0,0,0,0,0
11271,600000.0,5,2.25,2980,7781,9,1580,1400,1960,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data_pre.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       ...
       'zipcode_98146', 'zipcode_98148', 'zipcode_98155', 'zipcode_98166',
       'zipcode_98168', 'zipcode_98177', 'zipcode_98178', 'zipcode_98188',
       'zipcode_98198', 'zipcode_98199'],
      dtype='object', length=102)

In [11]:

scales_columns = {}
for col in data_pre.columns:
    mu = tr_df[col].mean()
    sigma = tr_df[col].std()
    scales_columns[col] = (mu,sigma)
    tr_df.loc[:,col] = tr_df.loc[:,col].apply(lambda x: (x - mu)/sigma)
    val_df.loc[:,col] = val_df.loc[:,col].apply(lambda x: (x - mu)/sigma)
    test_df.loc[:,col] = test_df.loc[:,col].apply(lambda x: (x - mu)/sigma)

x_tr = tr_df.drop(columns=['price'])
y_tr = tr_df['price']

x_val = val_df.drop(columns=['price'])
y_val = val_df['price']

x_test = test_df.drop(columns=['price'])
y_test = test_df['price']

/home/srodriguez/miniconda/envs/jupyterhub/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
x_tr.isna().values.astype(np.uint8).sum(axis=0).argmax()

0

In [14]:
x_tr.iloc[:,28:31].head()

,condition_3,condition_4,condition_5
17649,-1.349534,-0.602909,3.404898
9742,-1.349534,-0.602909,3.404898
4613,-1.349534,1.658471,-0.293667
19347,-1.349534,1.658471,-0.293667
11271,0.740928,-0.602909,-0.293667


In [15]:
def inverse_transform(data,column):
    mu, sigma = scales_columns[column]
    return (data*sigma) + mu

In [16]:
with dview.sync_imports(): #para todos los nodos dentro de la vista (dview) importamos las librerias relevantes
    from sklearn.svm import SVR #svr
    from sklearn import metrics #metricas para obtener el MSE

dview['x_tr'] = x_tr.values #Distribuimos a los nodos de la vista, el arreglo x_tr
dview['y_tr'] = y_tr.values #y_tr
dview['x_val'] = x_val.values #x_val
dview['y_val'] = y_val.values #y_val

dview['inverse_transform'] = inverse_transform #la funcion para revertir el scaling
dview['scales_columns'] = scales_columns # mu y sigmas para cada variable

Cs, epsilons = pd.np.meshgrid(np.logspace(-2,2,20),[i/40 for i in range(1,21)]) #generamos nuestra grilla
c_y_epsilon = list(zip(Cs.reshape((-1,1)), epsilons.reshape((-1,1)))) #la generamos como una lista

#y hacemos un scatter, esto implica que para cada nodo, solo tendra que computar una parte de la grilla
dview.scatter('c_y_epsilon',c_y_epsilon) 

#Definimos la funcion gridsearch que va a hacer la consulta de las estructuras dentro de cada nodo en el cluster:
#   - c_y_epsilon
#   - SVR (generar el regresor) y entrenarlo, predecir
#   - metrics.mean_squared_error Obtener el MSE
#   - inverse_transform (para hacer la transformación inversa)
#   - scales_columns dentro de 'inverse_transform' para poder 
# Si no estuviesen estas estructuras, el codigo se caeria 
# Esto tiene la misma estructura que la funcion iterativa, solo que la grilla esta distribuida
def gridsearch():
    results = []
    for C, epsilon in c_y_epsilon:
        sv = SVR(C=C[0], epsilon=epsilon[0])
        sv.fit(x_tr,y_tr)
        pred = sv.predict(x_val)
        mse = metrics.mean_squared_error(inverse_transform(y_val,'price'),inverse_transform(pred,'price'))
        results.append({'C':C[0],'epsilon':epsilon[0],'mse':mse})
    return results

importing SVR from sklearn.svm on engine(s)
importing metrics from sklearn on engine(s)


In [17]:
ar = dview.apply_async(gridsearch) #Ejecutamos la funcion gridsearch en cada nodo, de forma asincrona
ar.wait_interactive() #y esperamos a que nos retorne los resultados
res = ar.get() #si todo esta bien, esto nos debe devolver un arreglo de arreglos, cada uno con un diccionario de la misma forma que el iterativo
r_aux = [] #recorremos la respuesta asincrona 
for el in res:
    for subel in el:
        r_aux.append(subel) #y aplanamos (flatten) las respuestas

results = r_aux #luego por conveniencia, le llamamos igual que results de la forma iterativa

  17/17 tasks finished after  855 s
done


In [ ]:
results = pd.DataFrame(results)

In [ ]:
svr_results = pd.crosstab(results['C'],results['epsilon'],values=results['mse'],aggfunc=np.mean)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(svr_results, cmap="YlGnBu",square=False,annot=True,linewidths=.5)
plt.show()

In [ ]:
svr_top10 = results.sort_values(['mse']).head(10)
svr_top10

In [ ]:
@bview.parallel()
def gridsearch2(c_y_epsilon):
    C = c_y_epsilon[0][0]
    epsilon = c_y_epsilon[1][0]
    print("defined C")
    sv = SVR(C=C, epsilon=epsilon)
    sv.fit(x_tr, y_tr)
    pred = sv.predict(x_val)
    mse = metrics.mean_squared_error(inverse_transform(y_val,'price'),inverse_transform(pred,'price'))
    results = {'C':C,'epsilon':epsilon,'mse':mse}
    return results

In [ ]:
Cs, epsilons = pd.np.meshgrid(np.logspace(-2,2,20),[i/40 for i in range(1,21)]) #generamos nuestra grilla
c_y_epsilon = list(zip(Cs.reshape((-1,1)), epsilons.reshape((-1,1)))) #la generamos como una lista


In [ ]:
c_y_epsilon[0]

In [ ]:
res = gridsearch2.map(c_y_epsilon[:])

In [ ]:
%%time
ans = res.get()

In [ ]:
results = pd.DataFrame(ans)

In [ ]:
svr_results = pd.crosstab(results['C'],results['epsilon'],values=results['mse'],aggfunc=np.mean)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(svr_results, cmap="YlGnBu",square=False,annot=True,linewidths=.5)
plt.show()

In [ ]:
svr_top10 = results.sort_values(['mse']).head(10)
svr_top10